# Robot Machine Learning

- Many robot predictive maintenance applications require being able to decide whether a new observation belongs to the same distribution as existing observations or should be considered as an outlier.
- More information may be found here: https://scikit-learn.org/stable/modules/outlier_detection.html

## Create Robot Models

- We will create one "nominal" robot and one "defective" robot.
- The defective robot will include a small offset at the TCP (e.g., imagine the tool is wearing out).

In [1]:
from pybotics.predefined_models import ur10
from pybotics.robot import Robot

nominal_robot = Robot.from_parameters(ur10())

defective_robot = Robot.from_parameters(ur10())
defective_robot.tool.position = [0.1, 0, 0]

## Create Database

- We need to "collect" a large amount of data for machine learning applications

In [19]:
import numpy as np
from pybotics.geometry import matrix_2_vector

joints = np.random.uniform(low=-np.pi, high=np.pi, size=(1000, 6))
nominal_poses = np.array(
    [matrix_2_vector(nominal_robot.fk(j)) for j in joints])
defective_poses = np.array(
    [matrix_2_vector(defective_robot.fk(j)) for j in joints])

inliers = np.hstack((joints, nominal_poses))
outliers = np.hstack((joints, defective_poses))
outliers = outliers[:20]

data = np.vstack((inliers, outliers))

truth = np.ones(len(data))
truth[-len(outliers):] = -1

## Outlier Detection

- The data contains outliers which are defined as observations that are far from the others (i.e., defective TCP). 
- Outlier detection estimators try to fit the regions where the training data is the most concentrated, ignoring the deviant observations.

In [22]:
from sklearn.neighbors import LocalOutlierFactor

# create classifier
clf = LocalOutlierFactor(contamination='auto')

# predict
y_pred = clf.fit_predict(data)

# compute accuracy
n_errors = (y_pred != truth).sum()
accuracy = (1 - n_errors / len(y_pred)) * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 98.04%
